In [1]:
import falcon
from datetime import datetime

In [2]:
nivel_S=512 #nivel de seguridad

# Generación de claves para CA
def generate_ca_keypair(nivel_S):
    private_key = falcon.SecretKey(nivel_S)
    public_key = falcon.PublicKey(private_key)
    return private_key, public_key

# Generación de claves para certificados
def generate_cert_keypair(nivel_S):
    private_key = falcon.SecretKey(nivel_S)
    public_key = falcon.PublicKey(private_key)
    return private_key, public_key

# Funciones de firma

def generate_signature(private_key, data):
    datos= bytes(data, 'utf-8')
    sk=private_key
    firma = sk.sign(datos)
    # Simula la generación de una firma utilizando la clave privada
    return firma

# Verificación de firma
def verify_signature(public_key, data, firma):
    pk=public_key
    datos= bytes(data, 'utf-8')
    return pk.verify(datos, firma)


In [3]:
# Crear Autoridad Certificadora (CA)
def Crear_Autoridad():
    nivel_S=512 
    ca_private_key, ca_public_key = generate_ca_keypair(nivel_S)
    print("Escriba nombre de la autoridad")
    nombre_Autoridad=input(str())
    print("Escriba el email")
    email           =input(str())
    print("Pais de origen")
    pais            =input(str())
    data_Autoridad={
        'Autoridad':nombre_Autoridad,
        'País':pais,
        'e-mail':email
    }
    #print(data_Autoridad,ca_private_key, ca_public_key)
    return data_Autoridad,ca_private_key, ca_public_key

In [4]:
# Solicitud de certificado
def Solicitud_Certificado():
    nivel_S=512
    cert_private_key, cert_public_key = generate_cert_keypair(nivel_S)
    print("Nombre solicitante")
    nombre=input(str)
    
    cert_data = {
    'subject': nombre,
    'public_key': cert_public_key}
    #print(cert_data)
    return cert_data,cert_private_key


In [5]:
def Generacion_Certificado_Autoridad(solicitud_Certificado,Autoridad):
    datos=Autoridad[0]
    llave_privada=Autoridad[1]
    # Firma del certificado por la CA
    certificado_datos=solicitud_Certificado
    certificado_datos['Fecha']=datetime(2025, 1, 1)
    certificado_firmado= generate_signature(llave_privada, str(certificado_datos))
    #print(certificado_firmado,certificado_datos)
    return certificado_firmado,certificado_datos

In [6]:
def Frame(mensaje,certificado,llave_privada):
    sk=llave_privada
    mensaje_=bytes(mensaje, 'utf-8')
    firma=sk.sign(mensaje_)
    frame={'Certificado_Datos':certificado[1],
           'Firma_del_Certificado':certificado[0],
           'Mensaje':mensaje,
           'Mensaje_firmado':firma}
    #print(frame)
    return frame

In [7]:
def Verificacion_Certificado(ca_public_key,Frame):
    certificado_datos=Frame['Certificado_Datos']
    firma_certificado=Frame['Firma_del_Certificado']
    mensaje=Frame['Mensaje']
    mensaje_firmado=Frame['Mensaje_firmado']
    llave_publica_solicitante=certificado_datos['public_key'] #Llave publica del que manda el msj
    # Verificar la firma de la CA en el certificado
    ca_verification_result = verify_signature(ca_public_key, str(certificado_datos),firma_certificado)
    if ca_verification_result:
        print("Firma de la CA verificada correctamente en el certificado.")
        fecha_expiracion=certificado_datos['Fecha']
        hoy = datetime.now()
        if hoy<fecha_expiracion:
            
            if verify_signature(llave_publica_solicitante,str(mensaje),mensaje_firmado):
                print("Firma del certificado verificada")
                return True
            
            
        else:
            print("El certificado ha expirado")
            return False
    else:
        print("Firma de la CA no válida en el certificado.")
        return False

In [8]:
Autoridad=Crear_Autoridad()

Escriba nombre de la autoridad
Autoridad
Escriba el email
ca@gmail.com
Pais de origen
Mx


In [9]:
sc,llave_privada=Solicitud_Certificado()

Nombre solicitante
<class 'str'>Eliu


In [10]:
GS=Generacion_Certificado_Autoridad(sc,Autoridad)

In [11]:
frame= Frame("mensaje",GS,llave_privada)

In [12]:
verificacion=Verificacion_Certificado(Autoridad[2],frame)

Firma de la CA verificada correctamente en el certificado.
Firma del certificado verificada
